"""
https://keras.io/api/data_loading/image/

## Data Augmentation

Data augmentation is a technique used to increase the size and diversity of a training dataset by applying random transformations to the original images. This helps to prevent overfitting and improve the generalization ability of the model.

In this section, we will define functions that perform common data augmentation techniques such as rotation, zooming, and horizontal flipping using Keras's `ImageDataGenerator`.
"""

## Libraries

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os

## Functions

In [ ]:
def augment_image(path, datagen, setupgen):
  """
  Applies data augmentation to a single image.

  Args:
    image_array: A numpy array representing the image.
    datagen: An instance of ImageDataGenerator for data augmentation with the parameters specified.
  Returns:
    A numpy array representing the augmented image.
  """


  img = load_img(path)  # this is a PIL image
  x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
  x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

  # the .flow() command below generates batches of randomly transformed images
  # and saves the results to the `preview/` directory
  i = 0
  for batch in datagen.flow(x,
                            batch_size  = setupgen['Batch Size'],
                            save_to_dir = setupgen['Save Directory'],
                            save_prefix = setupgen['File Prefix'],
                            save_format = setupgen['File Format'] ):
      i += 1
      if i > 20:
          break  # otherwise the generator would loop indefinitely
  return


## Define data augmentation parameters

#### Data File Setup

In [ ]:
setupgen = {}
setupgen['File Format']     = "jpg" # @param ["png", "jpg", "jpeg", "bmp", "ppm", "tif", "tiff"]
setupgen['Save Directory']  = "DataAugmentation"
setupgen['File Prefix']     = ""
setupgen['Batch Size']      = 1


#### Data Gen Setup

In [ ]:
set_rotation_rng        = 45 # @param {type:"slider", min:0, max:180, step:5}
set_width_shift_rng     = 0.2 # @param {type:"slider", min:0, max:1, step:0.05}
set_height_shift_rng    = 0.2 # @param {type:"slider", min:0, max:1, step:0.05}
set_shear_rng           = 0.2 # @param {type:"slider", min:0, max:1, step:0.05}
set_zoom_rng            = 0.2 # @param {type:"slider", min:0, max:1, step:0.05}
set_horizontal_flip     = True # @param {type: "boolean"}
"""
The fill_mode parameter in ImageDataGenerator accepts the following values:
"nearest":  Fills the missing pixels with the nearest pixel value.
"reflect":  Fills the missing pixels with a reflection of the pixels near the edge.
"wrap":     Fills the missing pixels by wrapping the image around.
"constant": Fills the missing pixels with a constant value (specified by the cval parameter).
"""
set_fill_mode           = "reflect" # @param {type: "string"} ["constant", "wrap", "nearest", "reflect"]
set_cval                = 0 # @param {type:"slider", min:0, max:255, step:1}

datagen = ImageDataGenerator(
    rotation_range      = set_rotation_rng,         # Rotate images by a random degree between -20 and 20
    width_shift_range   = set_width_shift_rng,
    height_shift_range  = set_height_shift_rng,
    shear_range         = set_shear_rng,
    zoom_range          = set_zoom_rng,             # Zoom in or out by a random factor between 0.9 and 1.1
    horizontal_flip     = set_horizontal_flip,      # Randomly flip images horizontally
    fill_mode           = set_fill_mode,            # Fill in missing pixels after transformations
    set_cval            = set_cval,                 # Value to use for constant fill mode

  )


## Examples